In [1]:
import pandas as pd
import json
from pathlib import Path
import numpy as np
import glob
import os
import cv2
from lets_plot import *
LetsPlot.setup_html()

Пути к исходным файлам с координатами (Deeplabcut output)

In [2]:
sor_ara_files = glob.glob(os.path.join("../2_input/coord_from_deeplabcut/sor_ara", "*.csv"))
sor_cae_files = glob.glob(os.path.join("../2_input/coord_from_deeplabcut/sor_cae", "*.csv"))

Функция считывает csv и объединяет их в список

In [3]:
def read_files(path):
    df_list = []

    for file in path:

        df = pd.read_csv(file, skiprows=1, header=[0, 1])  # индексы столбцов иерархические, лежат в 2 строках

        # переделаем в длинный формат
        df = (
            df.iloc[:, 1:]  # первый столбец не нужен совсем
            .stack(level=[0], future_stack=True)
            .reset_index()
            .rename(columns={"level_0": "frame", "level_1": "bodyparts"})
        )

        df["ID"] = os.path.basename(file)[:-4]  # имя файла в качестве ID, но без расширения (.csv)

        df = df[["frame", "ID", "bodyparts", "x", "y", "likelihood"]]

        df_list.append(df)

    return df_list

Считать все файлы по видам и слить в датафрейм

In [4]:
sor_ara_data = pd.concat(read_files(sor_ara_files))
sor_cae_data = pd.concat(read_files(sor_cae_files))

Загрузим координаты поля (отступили 3 см вовнутрь от границ дна)

In [5]:
sor_ara_rectangle = pd.read_csv('../3_output/coordinates/square/sor_ara.csv')
sor_cae_rectangle = pd.read_csv('../3_output/coordinates/square/sor_cae.csv')

Объединим в 1 датафрейм

In [6]:
sor_ara_with_rectangle = sor_ara_data.merge(sor_ara_rectangle, how='left', on = 'ID')
sor_ara_with_rectangle['Spec'] = 'araneus'
sor_cae_with_rectangle = sor_cae_data.merge(sor_cae_rectangle, how='left', on = 'ID')
sor_cae_with_rectangle['Spec'] = 'caecutiens'
total = pd.concat([sor_ara_with_rectangle, sor_cae_with_rectangle])

In [7]:
def is_point_outside_rectangle(x, y, rectangle_points):
    """
    Проверяет, находится ли точка (x,y) ВНЕ прямоугольника.
    
    Параметры:
        x, y : координаты точки
        rectangle_points : список вершин прямоугольника [(x1,y1), (x2,y2), ...] (по часовой стрелке)
    
    Возвращает:
        1 если точка снаружи, 0 если внутри/на границе
    """
    # Преобразуем точки в numpy массив
    pts = np.array(rectangle_points, dtype=np.float32)
    
    # Вычисляем площадь прямоугольника (для проверки порядка точек)
    area = cv2.contourArea(pts)
    if area < 0:
        pts = pts[::-1]  # Если точки против часовой, разворачиваем
    
    # Проверяем, где находится точка
    result = cv2.pointPolygonTest(pts, (x, y), measureDist=False)
    
    return 1 if result < 0 else 0  # 1 = снаружи, 0 = на границе, 1 = внутри




# Функция для обработки каждой строки
def check_point_position(row):
    # Получаем вершины прямоугольника для текущего ID
    rect_points = [
        (row['L_top_x'], row['L_top_y']),
        (row['R_top_x'], row['R_top_y']),
        (row['R_bottom_x'], row['R_bottom_y']),
        (row['L_bottom_x'], row['L_bottom_y'])
    ]
    # Проверяем положение точки (x,y)
    return is_point_outside_rectangle(row['x'], row['y'], rect_points)

In [8]:
# Добавляем столбец с флагом
total['outside'] = total.apply(check_point_position, axis=1)

Теперь проверим, находится ли точка на крышке банки или вокруг банки

In [9]:
# загрузим json c координатами крышек и пространства вокруг банок
import json
import glob

# Путь к папке с JSON-файлами (можно использовать маску, например '*.json')
json_files = glob.glob('../3_output/coordinates/ovals/*.json')
json_files


['../3_output/coordinates/ovals\\sor_ara_ovals_left_side.json',
 '../3_output/coordinates/ovals\\sor_ara_ovals_right_side.json',
 '../3_output/coordinates/ovals\\sor_cae_ovals_left_side.json',
 '../3_output/coordinates/ovals\\sor_cae_ovals_right_side.json']

In [10]:

# Список для хранения данных из всех файлов
combined_data = []
# Чтение каждого файла и добавление данных в список
for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        # Читаем построчно, так как каждый JSON-объект на новой строке
        for line in f:
            if line.strip():  # Пропускаем пустые строки
                data = json.loads(line)
                combined_data.append(data)

# Сохранение объединенных данных в новый JSON-файл
output_file = '../3_output/coordinates/all_ovals.json'
with open(output_file, 'w', encoding='utf-8') as f:
    for item in combined_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"Объединено {len(json_files)} файлов в {output_file}. Всего объектов: {len(combined_data)}")

Объединено 4 файлов в ../3_output/coordinates/all_ovals.json. Всего объектов: 62


In [11]:
# 1. Загружаем объединенный JSON
data = []
with open('../3_output/coordinates/all_ovals.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Преобразуем JSON в словарь для быстрого доступа по ID
json_dict = {}
for item in data:
    json_dict[(item['ID'], item['side'])] = item  # Ключ — кортеж (ID, side)


In [12]:
# Функция для проверки точки в полигоне
def is_inside(point, polygon):
    if not polygon or len(polygon) < 3:  # Если полигон не валиден
        return False
    return cv2.pointPolygonTest(np.array(polygon, dtype=np.float32), (float(point[0]), float(point[1])), False) >= 0


In [13]:
def update_polygon_flags(row):
    item_L = json_dict.get((row['ID'], 'L'), {})
    item_R = json_dict.get((row['ID'], 'R'), {})
    
    x, y = row['x'], row['y']
    point = (x, y)

    in_inner_L = 1 if is_inside(point, item_L.get('inner', [])) else 0
    in_outer_only_L = 1 if (is_inside(point, item_L.get('outer', [])) and not in_inner_L) else 0

    in_inner_R = 1 if is_inside(point, item_R.get('inner', [])) else 0
    in_outer_only_R = 1 if (is_inside(point, item_R.get('outer', [])) and not in_inner_R) else 0

    return pd.Series({
        'in_inner_L': in_inner_L,
        'in_inner_R': in_inner_R,
        'in_outer_only_L': in_outer_only_L,
        'in_outer_only_R': in_outer_only_R,
    })

# Применяем к DataFrame
total[['in_inner_L', 'in_inner_R', 'in_outer_only_L', 'in_outer_only_R']] = total.apply(update_polygon_flags, axis=1)

А теперь преобразуем датафрейм для более удобной визуализации: удалим лишние столбцы, переименуем столбцы. Кроме того, левая и правая сторона не всегда соответствуют нахождению зверька в банке. Поэтому нужно создать столбцы отдельно: не левый/правый, а со зверьком/без зверька

In [14]:
total.drop(columns = ['L_top_x', 'L_top_y', 'R_top_x', 'R_top_y', 'L_bottom_x', 'L_bottom_y','R_bottom_x', 'R_bottom_y'], inplace = True)

In [15]:
# в этих случаях зверьки сидели в правой банке
total_without_cae_80_84 = total.query('ID != "Sar_84_13_07_24 00_00_10-00_05_10" & ID != "Sar_80_14_07_24 00_00_08-00_05_08"')

total_without_cae_80_84 = total_without_cae_80_84.rename(columns = {'outside': 'border',
                                          'in_inner_L': 'empty_cup',
                                          'in_outer_only_L': 'around_empty',
                                          'in_inner_R': 'with_sorex_cup',
                                          'in_outer_only_R':'around_with_sorex'})
total_without_cae_80_84 = total_without_cae_80_84.reindex(columns=['frame', 'ID', 'Spec', 'bodyparts', 'x', 'y', 'likelihood', 'border',
       'with_sorex_cup', 'around_with_sorex', 'empty_cup', 'around_empty'])

total_without_cae_80_84

,frame,ID,Spec,bodyparts,x,y,likelihood,border,with_sorex_cup,around_with_sorex,empty_cup,around_empty
0,0,Sar_203_13_07_24 00_00_09-00_05_09,araneus,nose,385.250519,464.600525,1.000000,0,0,0,0,0
1,0,Sar_203_13_07_24 00_00_09-00_05_09,araneus,head_center,391.422638,449.312775,0.981459,0,0,0,0,0
2,0,Sar_203_13_07_24 00_00_09-00_05_09,araneus,head_left,396.548401,452.100769,0.879107,0,0,0,0,0
3,0,Sar_203_13_07_24 00_00_09-00_05_09,araneus,head_right,385.472015,447.200470,0.864251,0,0,0,0,0
4,0,Sar_203_13_07_24 00_00_09-00_05_09,araneus,tail_base,402.748749,407.939301,1.000000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
630529,7508,Scaecut_801_23_07_2024 00_00_11-00_05_11,caecutiens,head_center,512.739563,75.459473,0.937790,1,0,0,0,0
630530,7508,Scaecut_801_23_07_2024 00_00_11-00_05_11,caecutiens,head_left,510.059235,71.614319,1.000000,1,0,0,0,0
630531,7508,Scaecut_801_23_07_2024 00_00_11-00_05_11,caecutiens,head_right,515.524597,78.958733,0.809778,1,0,0,0,0
630532,7508,Scaecut_801_23_07_2024 00_00_11-00_05_11,caecutiens,tail_base,496.787689,84.168480,0.947972,1,0,0,0,0


In [16]:
# для этих двух зверьков зверьки сидели в левой банке
change_cup = total.query(
    'ID == "Sar_84_13_07_24 00_00_10-00_05_10" | ID == "Sar_80_14_07_24 00_00_08-00_05_08"'
)


change_cup = change_cup.rename(
    columns={
        "outside": "border",
        "in_inner_L": "with_sorex_cup",
        "in_outer_only_L": "around_with_sorex",
        "in_inner_R": "empty_cup",
        "in_outer_only_R": "around_empty",
    }
)
change_cup = change_cup.reindex(columns=['frame', 'ID', 'Spec', 'bodyparts', 'x', 'y', 'likelihood', 'border',
       'with_sorex_cup', 'around_with_sorex', 'empty_cup', 'around_empty'])

change_cup

,frame,ID,Spec,bodyparts,x,y,likelihood,border,with_sorex_cup,around_with_sorex,empty_cup,around_empty
405222,0,Sar_80_14_07_24 00_00_08-00_05_08,araneus,nose,397.656600,61.615265,1.000000,1,0,0,0,0
405223,0,Sar_80_14_07_24 00_00_08-00_05_08,araneus,head_center,410.930500,61.083107,0.810036,1,0,0,0,0
405224,0,Sar_80_14_07_24 00_00_08-00_05_08,araneus,head_left,411.398930,71.411070,0.794283,0,0,0,0,0
405225,0,Sar_80_14_07_24 00_00_08-00_05_08,araneus,head_right,410.518740,54.784233,0.855882,1,0,0,0,0
405226,0,Sar_80_14_07_24 00_00_08-00_05_08,araneus,tail_base,443.363620,61.825650,0.994302,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
585265,7500,Sar_84_13_07_24 00_00_10-00_05_10,araneus,head_center,478.094421,456.566956,0.816552,0,0,0,0,1
585266,7500,Sar_84_13_07_24 00_00_10-00_05_10,araneus,head_left,473.725647,458.266632,0.856456,0,0,0,0,1
585267,7500,Sar_84_13_07_24 00_00_10-00_05_10,araneus,head_right,484.048218,455.950104,0.803040,0,0,0,0,1
585268,7500,Sar_84_13_07_24 00_00_10-00_05_10,araneus,tail_base,481.270081,499.418304,1.000000,1,0,0,0,0


In [17]:
# проверим, что размеры отфильтрованных датасетов в сумме равны исходному
change_cup.shape[0] + total_without_cae_80_84.shape[0] == total.shape[0]

True

In [18]:
result = pd.concat([total_without_cae_80_84, change_cup], axis = 0)

In [19]:
# добавим столбец, обозначающий, что точка находится где-то внутри (но не в любой из специально выделенных зон)
result["other"] = (
    (result["border"] != 1)
    & (result["with_sorex_cup"] != 1)
    & (result["around_with_sorex"] != 1)
    & (result["empty_cup"] != 1)
    & (result["around_empty"] != 1)
).astype(int)

In [20]:
# В ID зверька оставим только его номер и вид (для удобства)
result['ID'] = result['ID'].str.split('_').str[0:2].str.join('_')


In [21]:
# нужно добавить столбец с секундами
result['sec'] = result['frame']//25+1
result = result.reindex(columns = ['sec', 'frame', 'ID', 'Spec', 'bodyparts', 'x', 'y', 'likelihood', 'border',
       'with_sorex_cup', 'around_with_sorex', 'empty_cup', 'around_empty', 'other'])
result

,sec,frame,ID,Spec,bodyparts,x,y,likelihood,border,with_sorex_cup,around_with_sorex,empty_cup,around_empty,other
0,1,0,Sar_203,araneus,nose,385.250519,464.600525,1.000000,0,0,0,0,0,1
1,1,0,Sar_203,araneus,head_center,391.422638,449.312775,0.981459,0,0,0,0,0,1
2,1,0,Sar_203,araneus,head_left,396.548401,452.100769,0.879107,0,0,0,0,0,1
3,1,0,Sar_203,araneus,head_right,385.472015,447.200470,0.864251,0,0,0,0,0,1
4,1,0,Sar_203,araneus,tail_base,402.748749,407.939301,1.000000,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585265,301,7500,Sar_84,araneus,head_center,478.094421,456.566956,0.816552,0,0,0,0,1,0
585266,301,7500,Sar_84,araneus,head_left,473.725647,458.266632,0.856456,0,0,0,0,1,0
585267,301,7500,Sar_84,araneus,head_right,484.048218,455.950104,0.803040,0,0,0,0,1,0
585268,301,7500,Sar_84,araneus,tail_base,481.270081,499.418304,1.000000,1,0,0,0,0,0


Для проверки верного определения принадлежности нужно визуализировать треки, раскрасив точки в различные цвета

In [22]:
import matplotlib.pyplot as plt


# Укажите путь к папке, куда сохранять графики
output_dir = "../3_output/track_and_zones/"
os.makedirs(output_dir, exist_ok=True)

# Группируем данные по ID и bodyparts
grouped = result.groupby(['ID', 'bodyparts'])

# Функция для определения цвета точки
def get_point_color(row):
    if row['border'] == 1:
        return 'black'  # Точка по границе поля (периметр + стенки)
    elif row['with_sorex_cup'] == 1:
        return 'red'   # Точка на крышке банки со зверьком
    elif row['around_with_sorex'] == 1:
        return 'green'    # Точка 3 см вокруг банки со зверьком
    elif row['empty_cup'] == 1:
        return 'blue'   # Точка на крышке пустой банки
    elif row['around_empty'] == 1:
        return 'yellow' # Точка вокруг пустой банки
    elif row['other'] == 1:
        return 'PowderBlue' # остальные точки, не входящие ни в одну из зон

# Проходим по каждой группе и сохраняем графики
for (id_val, bodypart), group in grouped:
    plt.figure(figsize=(10, 8))
    
    # Инвертируем ось y для соответствия OpenCV/DLC
    y_inverted = group['y'].max() - group['y']  # Преобразование y
    
    # Применяем цвета к точкам
    colors = group.apply(get_point_color, axis=1)
    
    # Рисуем scatterplot с инвертированной осью y
    plt.scatter(
        group['x'], 
        y_inverted,  # Используем инвертированные y
        c=colors, 
        alpha=0.6, 
        edgecolors='w', 
        linewidth=0.5
    )
    
    # Добавляем легенду
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label='Стенка поля', markerfacecolor='black', markersize=10),
        plt.Line2D([0], [0], marker='o', color='w', label='Крышка (со зверьком)', markerfacecolor='red', markersize=10),
        plt.Line2D([0], [0], marker='o', color='w', label='Вокруг банки со зверьком', markerfacecolor='green', markersize=10),
        plt.Line2D([0], [0], marker='o', color='w', label='Крышка (пустая)', markerfacecolor='blue', markersize=10),
        plt.Line2D([0], [0], marker='o', color='w', label='Вокруг пустой банки', markerfacecolor='yellow', markersize=10),
        plt.Line2D([0], [0], marker='o', color='w', label='Все остальное', markerfacecolor='PowderBlue', markersize=10)
    ]
    
    plt.legend(handles=legend_elements, loc='upper right')
    plt.title(f"ID: {id_val}, Bodypart: {bodypart}")
    plt.xlabel("X")
    plt.ylabel("Y")  # Подпись с учётом инверсии
    plt.grid(True, alpha=0.3)
    
    # Сохраняем график
    filename = f"ID_{id_val}_Bodypart_{bodypart}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=150, bbox_inches='tight')
    plt.close()

С определением зон, судя по графикам, все более-мене в порядке, поэтому сохраним сырые данные, будем обрабатывать их уже в другом блокноте

In [23]:
result.to_csv('../3_output/row_data.csv')